In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from bs4 import BeautifulSoup as bs
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import time
import math
import json

In [3]:
original_URL = r"https://www.universityadmissions.se/intl/search?period=18&freeText="
#first_1000_URL = "https://www.universityadmissions.se/intl/search?period=18&sortBy=nameAsc&numberOfFetchedPages=20"

page = requests.get(original_URL)
soup = bs(page.content, "html.parser")

#print(soup.prettify())

### Hur många sidor finns det?

In [17]:
num_search_results = int(soup.find("div", class_="searchresult_summary universal_medium").text.strip().split(" ")[1].replace(",",""))
print(f"Number of search results: {num_search_results}")

# number of pages with 50 results each
import math
num_pages = math.ceil(num_search_results/50)
print(f"Number of pages: {num_pages}")


Number of search results: 4160
Number of pages: 84


## Skrapa hem informationen och spara i en dictionary

In [92]:
course_dict = {}

# Loopa igenom alla sidor om 50 resultat per sida
for page_num in range(num_pages):
    #print(f"Working with page {page_num}...")

    # Denna URL är för HT2023
    URL = f"https://www.universityadmissions.se/intl/search?period=18&freeText=&page={page_num+1}"
    page = requests.get(URL)
    soup = bs(page.content, "html.parser")

    # Hitta alla kurser/program
    search_result = soup.find("div", class_="searchresult")
    search_result = search_result.find("div", class_="resultsection")
    search_result_cards = search_result.find_all("div", class_="searchresultcard")

    # Loopa igenom varje kurs/program
    for tmp_course_idx, search_result_card in enumerate(search_result_cards):
        course_idx = 50*page_num + tmp_course_idx

        # Skapa en ny dictionary för varje kurs/program
        course_dict[course_idx] = {
            "course_name": "",
            "course_credits": "",
            "course_uni": "",
            "course_location": "",
            "application_status": "",
            "initial_tuition_fee": "",
            "total_tuition_fee": "",
            "period": "",
            "course_dates": "",
            "level": "",
            "language_of_instruction": "",
            "application_code": "",
            "teaching_form": "",
            "meetings": "",
            "pace_of_study": "",
            "instructional_time": "",
            "location": "",
            "meetings": "",
            "subject_areas": "",
            "link": "",
            "course_starts": "",
        }

        #print(f"Working with course {course_idx}...")

        # Header med kursnamn, hp, uni, location, application status
        resultcard_header = search_result_card.find("div", class_="resultcard_header")
        course_name = resultcard_header.find("h3").text
        course_dict[course_idx]["course_name"] = course_name

        course_info = resultcard_header.find("p").text.replace("\n", " ").split(",")
        course_credits = course_info[0].strip().split(" ")[0]
        course_uni = course_info[1].strip()
        course_location = course_info[2].strip().split(" ")[1].strip()

        course_dict[course_idx]["course_credits"] = course_credits
        course_dict[course_idx]["course_uni"] = course_uni
        course_dict[course_idx]["course_location"] = course_location

        application_status = resultcard_header.find("div", class_="applicable_status universal_medium").find("p")
        course_dict[course_idx]["application_status"] = application_status.text


        # Tuition fees
        tuition_fees = search_result_card.find("div", class_="neutralinfoblock infoblock").find_all("p")
        initial_tuition_fee = tuition_fees[1].text.split(":")[1].strip().strip(" SEK*").replace(",", "")
        total_tuition_fee = tuition_fees[2].text.split(":")[1].strip().strip(" SEK*").replace(",", "")

        course_dict[course_idx]["initial_tuition_fee"] = initial_tuition_fee
        course_dict[course_idx]["total_tuition_fee"] = total_tuition_fee


        # Kursdetaljer
        course_details_block = search_result_card.find("div", class_="course_details block")

        for row in course_details_block.find_all("p"):
            text = row.text.strip("\n")
            variable = text.split(":")[0].strip("\n").lower().replace(" ", "_")
            value = text.split(":")[1].strip("\n").replace("\n", " ")

            course_dict[course_idx][variable] = value


        # Subject areas
        resultcard_expanded = search_result_card.find("div", class_="resultcard_expanded universal_high")
        try:
            subject_areas = resultcard_expanded.find_all("p", class_="block")[1].contents[2].text.strip("\n")
        except Exception as e:
            subject_areas = ""
        course_dict[course_idx]["subject_areas"] = subject_areas


        # Link
        link = search_result_card.find("a", class_="external")["href"]
        course_dict[course_idx]["link"] = link




## Gör en dataframe av dictionaryn

In [106]:
# Gör en dataframe av kurserna
df = pd.DataFrame.from_dict(course_dict, orient='index')
print(f"Shape of dataframe: {df.shape}")
df.head()

Shape of dataframe: (4160, 20)


,course_name,course_credits,course_uni,course_location,application_status,initial_tuition_fee,total_tuition_fee,period,course_dates,level,language_of_instruction,application_code,teaching_form,meetings,pace_of_study,instructional_time,location,subject_areas,link,course_starts
0,3D Image Reconstruction and Analysis in Medicine,9,KTH Royal Institute of Technology,Huddinge,Open for late application,23250,23250,Autumn 2023 Period 1,28 Aug - 21 Jan,Master's,English,KTH-10154,"Course, On-campus",,33%,Daytime,Huddinge,Medical Technologies,https://www.kth.se/student/kurser/kurs/HL2027?...,
1,A History of Mathematics for Teachers,7.5,Dalarna University,Falun,Open for late application,16875,16875,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,English,HDA-H3EV4,"Course, On-campus",,Half-time,Daytime,Falun,Mathematics,http://www.du.se/redirect/course.aspx?language...,
2,A History of Mathematics for Teachers,7.5,Dalarna University,Falun,Open for late application,16875,16875,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,English,HDA-H3EV6,"Course, Distance",4,Half-time,Daytime,Falun,Mathematics,http://www.du.se/redirect/course.aspx?language...,
3,A New Heaven and a New Earth,7.5,Newmaninstitutet,Uppsala,Closed for late application,Information not available,12500,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,English,NMI-L0362,"Course, On-campus",,Half-time,Daytime,Uppsala,Humanities,https://newman.se/kurser-ht-2023/#a-new-heaven...,
4,A Theme from the History of Theoretical Philos...,7.5,Stockholm University,Stockholm,Open for late application,11250,11250,Autumn 2023 Period 1,28 Aug - 29 Oct,Bachelor's,English,SU-10924,"Course, On-campus",,Half-time,Daytime,Stockholm,Philosophy and Logic,https://sisu.it.su.se/search/info/FITE10/en?ev...,


## Information om "subjects" på olika nivåer

Det verkar som att man måste klicka på "Subjects" för att få ut HTML-koden som representerar de olika filteralternativen. Annars finns inte information tillgänglig. 
<br><br>
Detta kan göras med selenium för att klicka på knappen och sedan spara HTML-koden och bearbeta den i BS4 som vanligt.

In [254]:
original_URL = r"https://www.universityadmissions.se/intl/search?period=18&freeText="

# Öppna hemsidan
driver = webdriver.Safari()
driver.maximize_window()
driver.get(original_URL)

time.sleep(2)

# Hitta knappen och klicka på den
expand_subject_filter_button = driver.find_element(By.XPATH, '//*[@id="searchform"]/div/div[2]/fieldset[2]/div[2]/div/button')
expand_subject_filter_button.click()

time.sleep(2)

# Spara ner HTML-koden för vidare bearbetning i BeautifulSoup
search_result = driver.find_element(By.XPATH, '//*[@id="subjecttree"]/div')
soup = bs(search_result.get_attribute("outerHTML"), "html.parser")

# Stäng webbläsaren
driver.close()

### Leta runt i listorna med BS4
Alla listor med subjects och underkategorier börjar med en ```"All ..."```.
<br><br>
Det kan finnas fler "grenar" med underkategorier för en underkategori. T.ex. fler level2 i en level1.

In [375]:
subjects = {}

level1 = soup.find("ul", class_="level1 list-body")
level1_items = level1.find_all("li", recursive=False)

for level1_item in level1_items[1:]:
    level1_subject = level1_item.find("span", class_="option-name").text.strip()
    subjects[level1_subject] = {}
    
    try:
        level2 = level1_item.find("ul", class_="level2 list-body")

        level2_items = level2.find_all("li", recursive=False)
        for level2_item in level2_items:
            level2_subject = level2_item.find("span", class_="option-name").text.strip().replace("\t", "").replace("\n", " ")
            subjects[level1_subject][level2_subject] = {}

            try:
                level3 = level2_item.find("ul", class_="level3 list-body")

                level3_items = level3.find_all("li", recursive=False)
                for level3_item in level3_items:
                    level3_subject = level3_item.find("span", class_="option-name").text.strip().replace("\t", "").replace("\n", " ")
                    subjects[level1_subject][level2_subject][level3_subject] = {}
            
            except Exception as e:
                #print(e)
                pass

    except Exception as e:
        pass






In [5]:
# Spara till fill
with open('subjects.json', "w", encoding="utf-8") as f:
    json.dump(subjects, f, indent=4)

#print(json.dumps(subjects, indent=4))

In [94]:
# Läs från fil
with open('subjects.json') as f:
    subjects = json.load(f)

print(json.dumps(subjects, indent=4))

{
    "Agriculture, Horticulture, Forestry and Fishery": {
        "All agriculture, horticulture, forestry and fishery": {},
        "Agricultural Science": {
            "All agricultural science": {},
            "Food Science": {}
        },
        "Agriculture and Forestry": {},
        "Animal Science": {
            "All animal science": {},
            "Equine Studies": {}
        },
        "Fishery": {},
        "Forestry": {},
        "Horticultural Science": {},
        "Soil Science": {}
    },
    "Arts, Design and Media": {
        "All arts, design and media": {},
        "Aesthetics": {},
        "Architecture": {
            "All architecture": {},
            "Landscape Architecture": {},
            "Lighting Design": {}
        },
        "Crafts": {},
        "Creative Writing": {},
        "Dance Studies": {},
        "Design": {},
        "Fashion Studies": {},
        "Film Studies": {},
        "Fine Arts": {},
        "History and Theory of Arts": {},
      

## Gör en mappning från djupaste sub-kategori till högsta kategori

### Vissa sub-kategorier finns i flera övre kategorier, så det matchas till en lista med huvudkategorier

In [95]:
# Denna kod är hämtad från GPT-4

def flatten_subjects(nested_dict, current_subject=None, result=None):
    if result is None:
        result = {}

    for key, value in nested_dict.items():
        if current_subject is not None:
            if key in result:
                if current_subject not in result[key]:
                    result[key].append(current_subject)
            else:
                result[key] = [current_subject]

        if isinstance(value, dict):
            # Continue to traverse deeper if dictionary is not empty
            if value:
                flatten_subjects(value, key if current_subject is None else current_subject, result)

    # Map top-level categories to themselves
    if current_subject is None:
        for key in nested_dict:
            if key not in result:
                result[key] = [key]

    return result

In [96]:
flattened_subjects = flatten_subjects(subjects)

# Spara till fill
with open('flattened_subjects.json', "w", encoding="utf-8") as f:
    json.dump(flattened_subjects, f, indent=4)


In [97]:
# Öppna json-filen med alla ämnen
with open('flattened_subjects.json') as f:
    flattened_subjects = json.load(f)

print(json.dumps(flattened_subjects, indent=4))

{
    "All agriculture, horticulture, forestry and fishery": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "Agricultural Science": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "All agricultural science": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "Food Science": [
        "Agriculture, Horticulture, Forestry and Fishery",
        "Materials, Construction and Manufacturing",
        "Natural Science"
    ],
    "Agriculture and Forestry": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "Animal Science": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "All animal science": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "Equine Studies": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "Fishery": [
        "Agriculture, Horticulture, Forestry and Fishery"
    ],
    "Forestry": [
        "Agriculture, Horticulture, Fores

## Gör en ny kolumn med huvudämnen för varje kurs/program

In [122]:
def map_top_level_subject(subject_areas):

    top_level_subjects = []

    for subject in subject_areas.split(","):
        top_level_subjects.append(flattened_subjects.get(subject.strip(), [subject.strip()]))

    top_level_subjects_cleaned = list(set(sum(top_level_subjects, [])))

    return " | ".join(top_level_subjects_cleaned)

In [123]:
# Gör en ny kolumn med top level subjects
df["top_level_subjects"] = df.subject_areas.apply(map_top_level_subject)
df.head()

,course_name,course_credits,course_uni,course_location,application_status,initial_tuition_fee,total_tuition_fee,period,course_dates,level,...,application_code,teaching_form,meetings,pace_of_study,instructional_time,location,subject_areas,top_level_subjects,course_starts,link
0,3D Image Reconstruction and Analysis in Medicine,9,KTH Royal Institute of Technology,Huddinge,Open for late application,23250,23250,Autumn 2023 Period 1,28 Aug - 21 Jan,Master's,...,KTH-10154,"Course, On-campus",,33%,Daytime,Huddinge,Medical Technologies,Health and Medical Care,,https://www.kth.se/student/kurser/kurs/HL2027?...
1,A History of Mathematics for Teachers,7.5,Dalarna University,Falun,Open for late application,16875,16875,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,...,HDA-H3EV4,"Course, On-campus",,Half-time,Daytime,Falun,Mathematics,Mathematics,,http://www.du.se/redirect/course.aspx?language...
2,A History of Mathematics for Teachers,7.5,Dalarna University,Falun,Open for late application,16875,16875,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,...,HDA-H3EV6,"Course, Distance",4,Half-time,Daytime,Falun,Mathematics,Mathematics,,http://www.du.se/redirect/course.aspx?language...
3,A New Heaven and a New Earth,7.5,Newmaninstitutet,Uppsala,Closed for late application,Information not available,12500,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,...,NMI-L0362,"Course, On-campus",,Half-time,Daytime,Uppsala,Humanities,Humanities,,https://newman.se/kurser-ht-2023/#a-new-heaven...
4,A Theme from the History of Theoretical Philos...,7.5,Stockholm University,Stockholm,Open for late application,11250,11250,Autumn 2023 Period 1,28 Aug - 29 Oct,Bachelor's,...,SU-10924,"Course, On-campus",,Half-time,Daytime,Stockholm,Philosophy and Logic,Humanities,,https://sisu.it.su.se/search/info/FITE10/en?ev...


In [124]:
# Funktion som byter plats på två kolumner i en dataframe

def swap_columns(df, column1, column2):
    cols = df.columns.tolist()
    a, b = cols.index(column1), cols.index(column2)
    cols[a], cols[b] = cols[b], cols[a]
    df = df[cols]
    return df

In [126]:
# Byt plats på två kolumner i en dataframe
df = swap_columns(df, 'top_level_subjects', 'link')
df.head()

,course_name,course_credits,course_uni,course_location,application_status,initial_tuition_fee,total_tuition_fee,period,course_dates,level,...,application_code,teaching_form,meetings,pace_of_study,instructional_time,location,subject_areas,top_level_subjects,course_starts,link
0,3D Image Reconstruction and Analysis in Medicine,9,KTH Royal Institute of Technology,Huddinge,Open for late application,23250,23250,Autumn 2023 Period 1,28 Aug - 21 Jan,Master's,...,KTH-10154,"Course, On-campus",,33%,Daytime,Huddinge,Medical Technologies,Health and Medical Care,,https://www.kth.se/student/kurser/kurs/HL2027?...
1,A History of Mathematics for Teachers,7.5,Dalarna University,Falun,Open for late application,16875,16875,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,...,HDA-H3EV4,"Course, On-campus",,Half-time,Daytime,Falun,Mathematics,Mathematics,,http://www.du.se/redirect/course.aspx?language...
2,A History of Mathematics for Teachers,7.5,Dalarna University,Falun,Open for late application,16875,16875,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,...,HDA-H3EV6,"Course, Distance",4,Half-time,Daytime,Falun,Mathematics,Mathematics,,http://www.du.se/redirect/course.aspx?language...
3,A New Heaven and a New Earth,7.5,Newmaninstitutet,Uppsala,Closed for late application,Information not available,12500,Autumn 2023 Period 1,28 Aug - 5 Nov,Bachelor's,...,NMI-L0362,"Course, On-campus",,Half-time,Daytime,Uppsala,Humanities,Humanities,,https://newman.se/kurser-ht-2023/#a-new-heaven...
4,A Theme from the History of Theoretical Philos...,7.5,Stockholm University,Stockholm,Open for late application,11250,11250,Autumn 2023 Period 1,28 Aug - 29 Oct,Bachelor's,...,SU-10924,"Course, On-campus",,Half-time,Daytime,Stockholm,Philosophy and Logic,Humanities,,https://sisu.it.su.se/search/info/FITE10/en?ev...


In [127]:
df.to_csv("antagningsstatistik.csv", index=False)
df.to_excel("antagningsstatistik.xlsx", index=False)

### Info och statistik om dataframen

In [144]:
df.describe()

,course_name,course_credits,course_uni,course_location,application_status,initial_tuition_fee,total_tuition_fee,period,course_dates,level,...,application_code,teaching_form,meetings,pace_of_study,instructional_time,location,subject_areas,top_level_subjects,course_starts,link
count,4160,4160,4160,4160,4160,4160,4160,4160,4160,4160,...,4160,4160,4160,4160,4160,4160,4160,4160,4160,4160
unique,3425,27,37,51,7,323,409,2,81,4,...,4141,6,32,12,6,52,950,245,10,3808
top,Master's Programme in Language Science,7.5,Stockholm University,Stockholm,Open for late application,11250,11250,Autumn 2023 Period 1,,Master's,...,,"Course, On-campus",,Full-time,Daytime,Stockholm,Biology,Natural Science,,http://smi.se/english/index.html
freq,24,1659,690,828,2028,177,177,3183,1518,2843,...,20,2051,3411,2452,3630,837,139,538,2715,17


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4160 entries, 0 to 4159
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   course_name              4160 non-null   object 
 1   course_credits           4160 non-null   float32
 2   course_uni               4160 non-null   object 
 3   course_location          4160 non-null   object 
 4   application_status       4160 non-null   object 
 5   initial_tuition_fee      4160 non-null   object 
 6   total_tuition_fee        4160 non-null   object 
 7   period                   4160 non-null   object 
 8   course_dates             4160 non-null   object 
 9   level                    4160 non-null   object 
 10  language_of_instruction  4160 non-null   object 
 11  application_code         4160 non-null   object 
 12  teaching_form            4160 non-null   object 
 13  meetings                 4160 non-null   object 
 14  pace_of_study            4160

### Manipulera dataframe

In [234]:
# Gör om course_credits (HP) till float32
df.course_credits = df.course_credits.astype("float32")

# Spara till filer
#df.to_csv("antagningsstatistik.csv", index=False)
#df.to_excel("antagningsstatistik.xlsx", index=False)

In [175]:
df.describe()

,course_credits
count,4160.000000
mean,45.387859
std,52.494816
min,0.000000
25%,7.500000
50%,15.000000
75%,120.000000
max,300.000000


### Omvandla tuition fee till tal?

In [221]:
df.initial_tuition_fee.unique()

array(['23250', '16875', 'Information not available', '11250', '21000',
       '8745', '11125', '11660', '21750', '46500', '45000', '19300',
       '48000', '17500', '38060', '88050', '18550', '100000', '37500',
       '18800', '15500', '31000', '35700', '14400', '12600', '21375',
       '19375', '74982', '17850', '11598', '11498', '35000', '20667',
       '18000', '18625', '71400', '23333', '12900', '127500', '17873',
       '63000', 'None', '22250', '1450', '11600', '13050', '17400',
       '40000', '15625', '15000', '56500', '13090', '62500', '67000',
       '75000', '7200', '10333', '19030', '57500', '12313', '38054',
       '12000', '67500', '10000', '74500', '60000', '10998', '77500',
       '80000', '13875', '12083', '18750', '26163', '36000', '13750',
       '32500', '105000', '23400', '93600', '70000', '72250', '18125',
       '150000', '62000', '9900', '5625', '9800', '51000', '50000',
       '130000', '145000', '119000', '95000', '20000', '90000', '43500',
       '120000', '

In [216]:
# Icke-numerisk data kan vara antingen "Information not available" eller "None"
rows_to_drop = df.loc[df.initial_tuition_fee.isin(["None", "Information not available"]) | df.total_tuition_fee.isin(["None", "Information not available"])].index
tuition_fees = df.drop(rows_to_drop)[["initial_tuition_fee", "total_tuition_fee"]]

# Konvertera till numerisk data
tuition_fees = tuition_fees.apply(pd.to_numeric, errors='coerce')

print(f"Shape of tuition_fees: {tuition_fees.shape}")
tuition_fees.head()

Shape of tuition_fees: (4083, 2)


,initial_tuition_fee,total_tuition_fee
0,23250,23250
1,16875,16875
2,16875,16875
4,11250,11250
5,21000,21000
